# Heatmaps at selected sites
This notebook generates heatmaps at single sites or ranges of sites. These are generated separately so that gaps can be inserted between non-continuous sites in the finalized figure. 

Heatmaps for escape from sera 3944 and 197C at selected sites are visualized in Figure 1. Max/min escape is capped at 1 (the approximate maximum escape score at these sites for these sera).

In [1]:
import pickle

import altair as alt
import altair_saver

import pandas as pd

import polyclonal

import warnings
warnings.filterwarnings('ignore')

import yaml

from IPython.utils import io

In [2]:
import os
os.chdir('../../')

### Get plotting parameters

In [3]:
# config parameters are the same for all sera
with open('data/polyclonal_config.yaml') as f:
    antibody_config = yaml.safe_load(f)['197C']

In [4]:
muteffects_csv = "results/muteffects_functional/muteffects_observed.csv"
site_numbering_map = "data/site_map.csv"
max_escape=1

site_map = pd.read_csv(site_numbering_map).rename(columns={"reference_site": "site"})

df_to_merge = [site_map]

plot_kwargs = antibody_config["plot_kwargs"]

muteffects = pd.read_csv(muteffects_csv).rename(
    columns={"reference_site": "site", "effect": "functional effect"}
)[["site", "mutant", "functional effect"]]
plot_kwargs["addtl_slider_stats"]["functional effect"] = -1.38

plot_kwargs["addtl_slider_stats_hide_not_filter"] = []

plot_kwargs["addtl_slider_stats_hide_not_filter"].append("functional effect")

plot_kwargs["init_floor_at_zero"] = False

plot_kwargs["alphabet"] = ['R', 'K', 'H', 'D', 'E', 'Q', 'N', 'S', 'T', 'Y', 
                           'W', 'F', 'A', 'I', 'L', 'M', 'V', 'G', 'P', 'C']

plot_kwargs["show_lineplot"] = False

df_to_merge.append(muteffects)

### Generate heatmaps

In [5]:
def expand_ranges(range_list):
    expanded_list = []
    for item in range_list:
        if isinstance(item, int):
            expanded_list.append(item)
        elif isinstance(item, str):
            if ':' in item:
                start, end = item.split(':')
                expanded_list.extend(range(int(start), int(end) + 1))
            else:
                expanded_list.append(int(item))
    return expanded_list

In [6]:
for antibody in [3944, '197C']:
    
    # get the polyclonal model for that antibody
    pickle_file=f'results/antibody_escape/{antibody}.pickle'
    with open(pickle_file, "rb") as f:
        model = pickle.load(f)
    f.close()
    
    # define sites
    site_ranges = ['121:124', 145, '158:160', '188:193']
    
    heatmaps = []
    
    # each entry is a block of sites to plot together
    for entry in site_ranges:
        sites = expand_ranges([entry])
        # get heatmap using defined parameters
        heatmap = model.mut_escape_plot(
            df_to_merge=df_to_merge,
            **plot_kwargs,
            sites_to_show={"include": sites},
            heatmap_max_fixed=max_escape,
            heatmap_min_fixed=-max_escape
        )
        
        # save as high-res png
        heatmap.save(f'figures/heatmaps/results/{antibody}_{entry}_heatmap.png', 
                     scale_factor=2.0)